[DIY Disease Tracking Dashboard Kit](https://github.com/weavermech/coda) (C) Stephen Hanson 2024 ([stephenhansonj@gmail.com](stephenhansonj@gmail.com))  
Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).


In [ ]:
from anytree import Node, RenderTree
from anytree.importer import JsonImporter, DictImporter
import datetime
from IPython.display import clear_output, Markdown
import ipywidgets as wdg
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import pickle
import requests
import time

import warnings
warnings.filterwarnings('ignore', category=FutureWarning) # avoiding pandas warning

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100
global regions 
regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
global filters
filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
    "age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
    "sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
    "year": None,#2022, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
    "month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
    "epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
    "date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
    "in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
}

In [ ]:
# loading data for initial graphs
with open("mean_occBed_region.json", "rt") as INFILE:
    mean_occBed_region=json.load(INFILE)

with open("leaf_first.json", "rt") as INFILE:
    leaf_data =json.load(INFILE)

In [ ]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # editied to include the geography
    data={}
    for dataset in rawdata:
        for entry in dataset:
            date=entry['date']
            value=entry['metric_value']
            geography=entry['geography']
            if date not in data:
                data[date]={}
            data[date][geography]=value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    mean_occBed_region_df=pd.DataFrame(index=index, columns=regions)

    for date, values in data.items():
        for region, value in values.items():
            mean_occBed_region_df.at[parse_date(date), region] = value
    # fill in any remaining "holes" due to missing dates
    mean_occBed_region_df.fillna(0.0, inplace=True)

    return mean_occBed_region_df


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df1=wrangle_data(mean_occBed_region) # df is the dataframe for plotting

In [ ]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    
    class APIwrapper:
        # class variables shared among all instances
        _access_point="https://api.ukhsa-dashboard.data.gov.uk/v2"       
        _last_access=0.0 # time of last api access
        root = Node("root")
        
        def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
            """ Init the APIwrapper object, constructing the endpoint from the structure
            parameters """
            # build the path with all the required structure parameters. You do not need to edit this line,
            # parameters will be replaced by the actual values when you instantiate an object of the class!
            url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                    f"{geography_type}/geographies/{geography}/metrics/{metric}")
            # our starting API endpoint
            self._start_url=APIwrapper._access_point+url_path
            self._filters=None
            self._page_size=-1
            # will contain the number of items
            self.count=None



        def get_page(self, filters={}, page_size=5):
            """ Access the API and download the next page of data. Sets the count
            attribute to the total number of items available for this query. Changing
            filters or page_size will cause get_page to restart from page 1. Rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365); use the default value 
            for debugging your structure and filters. """
            # Check page size is within range
            if page_size>365:
                raise ValueError("Max supported page size is 365")
            # restart from first page if page or filters have changed
            if filters!=self._filters or page_size!=self._page_size:
                self._filters=filters
                self._page_size=page_size
                self._next_url=self._start_url
            # signal the end of data condition
            if self._next_url==None: 
                return [] # we already fetched the last page
            # simple rate limiting to avoid bans
            curr_time=time.time() # Unix time: number of seconds since the Epoch
            deltat=curr_time-APIwrapper._last_access
            if deltat<0.33: # max 3 requests/second
                time.sleep(0.33-deltat)
            APIwrapper._last_access=curr_time
            # build parameter dictionary by removing all the None
            # values from filters and adding page_size
            parameters={x: y for x, y in filters.items() if y!=None}
            parameters['page_size']=page_size
            # the page parameter is already included in _next_url.
            # This is the API access. Response is a dictionary with various keys.
            # the .json() method decodes the response into Python object (dictionaries,
            # lists; 'null' values are translated as None).
            response = requests.get(self._next_url, params=parameters).json()
            # update url so we'll fetch the next page
            self._next_url=response['next']
            self.count=response['count']
            # data are in the nested 'results' list
            return response['results'] 

        def get_all_pages(self, filters={}, page_size=365):
            """ Access the API and download all available data pages of data. Sets the count
            attribute to the total number of items available for this query. API access rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365), and controls the trade-off
            between time to load a page and number of pages; the default should work well 
            in most cases. The number of items returned should in any case be equal to 
            the count attribute. """
            data=[] # build up all data here
            while True:
                # use get_page to do the job, including the pacing
                next_page=self.get_page(filters, page_size)
                if next_page==[]:
                    break # we are done
                data.extend(next_page)
            return data


        # /themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/{geography_type}/geographies/{geography}/metrics/{metric}
        def add_branch_to_node(self, upper_node=None, link=None):
        
            # to kick off the tree from scratch
            # had some problems with using defualt values in the function so dealing with it here
            if upper_node is None:
                upper_node = APIwrapper.root
            if link is None:
                link = APIwrapper._access_point + '/themes/'
                
            importer = DictImporter()
            curr_time = time.time()  # Unix time: number of seconds since the Epoch
            deltat = curr_time - APIwrapper._last_access
            if deltat < 0.33:  # max 3 requests/second
                time.sleep(0.33 - deltat)
            APIwrapper._last_access = curr_time

            branches = requests.get(link).json()
            if branches:                                                # make sure the branch isn't empty                          
                for branch in branches:
                    new_node = importer.import_(branch)                 # create a new node from the branch using the dictionary importer from anytree
                    new_node.parent = upper_node
                    new_node.name = branch['name']                      # Use the value of the "name" dictionary key

                    curr_time = time.time()                             # Unix time: number of seconds since the Epoch
                    deltat = curr_time - APIwrapper._last_access
                    if deltat < 0.33:                                   # max 3 requests/second
                        time.sleep(0.33 - deltat)
                    APIwrapper._last_access = curr_time
                    next_level_link = branch['link']                    # get the link to the next section in the api

                    if "/metrics/" in next_level_link:                  # hacky way to get to find if we are at the bottom of the tree (metric data is next)
                        continue                                        # continue to next branch

                    options_link = requests.get(next_level_link).json() # get the link that returns the options for the next branch
                    options_link = list(options_link[0].values())[0]
                    
                    self.add_branch_to_node(new_node, options_link)     # recursively add the next branch to the tree

            return
    
    # This Next Line Builds The Tree  --  !!It Takes Over 50min To Run on Local Machine So It Is Commented Out And Runs From File!!
    # api.add_branch_to_node()
    
    # Load the tree from a pickle file instead
    global root
    with open('full_tree_named.pkl', 'rb') as f:
        root = pickle.load(f)

    structure={"theme": "infectious_disease", 
        "sub_theme": "respiratory",
        "topic": "COVID-19", "metric": "COVID-19_healthcare_occupiedBedsRollingMean", "geography_type": "NHS Region"} # leave out geography, will be dynamic
    
    # build the raw data
    mean_occBed_region = []
    for idx, r in enumerate(regions):
        structure["geography"] = r
        api=APIwrapper(**structure)
        mean_occBed_region.append(api.get_all_pages(filters))                    
        print(f"Data points expected: {api.count}")
        print(f"Data points retrieved: {len(mean_occBed_region[idx])}")

    # is a one-off save to create the file. Keep for future reference though
    # with open("mean_occBed_region.json", "wt") as OUTF:
    #     json.dump(mean_occBed_region, OUTF)
        
    return mean_occBed_region

In [ ]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.

    global df1, df2
    global regions 
    regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
    print("getting data")
    try:
        apidata=access_api()
        df1=wrangle_data(apidata)
        refresh_graph()
        apibutton.icon="check"
        apibutton.description = 'Success'
        apibutton.style.button_color = 'green'
        apibutton.disabled=True

    except Exception as err:
        apibutton.icon = "unlink"
        apibutton.description = "Try Again"
        apibutton.style.button_color = 'red'

    
apibutton=wdg.Button(
    description='Refresh', 
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    icon='solid download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

# line below commented to display later
# display(apibutton)


## First Interactive Graph - Mean Beds Occupied by Region and Time Period 

This graph displays regional data for the mean number of beds occupied. The data for 7 English regions from Aug 2020 to Oct 2024 are availible. The initial graph is from data saved on 25th November 2024. 
  
- Use the multi-select list to select regions (hold shift or ctrl and click to select multiple). The graph dymamically updates as the region list is changed.  
- Use the date sliders to specify the start and end dates for the plot. The graph updates as the start and end dates are changed, and the y-axis scales appropriately for the data.
- Click the refresh button to use data from the API - it takes a few seconds to download the 4 years of data for all regions. The checkmark will display when the download is successful, the button will turn green and be disabled.
    - If the data is not availible, the button will turn red and display "Try again" text and a broken icon. The button remains enabled and can be reclicked to retry the download.

In [ ]:
def update_plot(selected_regions, date_range):
    fig, ax = plt.subplots(figsize=(12, 6))
    selected_regions = list(selected_regions)                   # make a list of the regions to filter by
    start_date, end_date = date_range                           # pull the start and end date out of the tuple              
    df1a = df1.loc[start_date:end_date, selected_regions].copy()               
    df1a.plot(ax=ax)
    ax.set_title("Mean Beds Occupied by Region")
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=75)
    ax.tick_params(axis='x', which='major', rotation=75)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    # started by filtering the visible plot, but since df is a reasonable size decided to filter data instead
    # keeping for future reference
    # ax.set_xlim([start_date, end_date])   
                        
    # since filtering date by the data, can scale the y-axis based on the date range
    ax.set_ylim([df1a.min().min(), df1a.max().max()+50])
    
    plt.show()
  
  
reg = wdg.SelectMultiple(
    options=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    value=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    description='Regions',
    disabled=False
)
reg.layout.height = '120px'


# slider is fixed for dates at this time. Would be interesting to change to dynamic, but out of the scope of this project
dates = pd.date_range(start='2020-08-01', end='2024-10-31', freq='MS')
options = [(i.strftime('%b %Y'), i) for i in dates]
dates_slider = wdg.SelectionRangeSlider(
    options=options,
    index=(0, 50),
    description='Period',
    disabled=False,
    layout=wdg.Layout(width='40%')
)

# forcably prevent the start date equalling the end date. Might be improved.
def validate_date_range(change):
    start, end = change['new']
    if start.year == end.year and start.month == end.month:
        if start == dates[0]:
            end = dates[1]
        else:
            start = dates[dates.get_loc(end) - 1]
        yr.value = (start, end)

dates_slider.observe(validate_date_range, names='value')



def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=reg.value
    if current==reg.options[0]:
        other=reg.options[1]
    else:
        other=reg.options[0]
    reg.value=(other,) # forces the redraw
    reg.value=current # now we can change it back
    
controls0=wdg.HBox([apibutton, reg, dates_slider]) 

# connect the plotting function and the widget    
graph0=wdg.interactive_output(update_plot, {'selected_regions': reg, 'date_range': dates_slider})

# actually displays the graph
display(controls0, graph0)

# Interactive Graph 2 - Access to the Full API Data

I'd heard of tree data stuctures a few years ago and decided to map the whole API to a tree structure. This took nearly an hour on my machine so it is written to a pickle file and loaded from there. The mapping code is included in the function add_branch_to_node.   
Each node has a dictionary containing the api link and the name, and the function explores all branches down to the metrics data.  
The code checks when a dropdown is changed and updates the following dropdowns so you can't have parent showing that doesn't belong to a child that is shown in another dropdown.  
I wanted to try pulling the data from the link in the Metrics dictionary, rather than building a link with the structure, so pages are retrieved outside the APIwrapper class and some functions are repeated for this stand alone graph.  
  
- The initial plot is the first leaf in the tree.
- Use a dropdown to choose a sub theme, topic, geography type, geography, or metric. The dropdowns that follow the one you changed will update with the available entries.
- Once you have built your selection, press the Refresh button:
    - If the refresh is successful the button changes to blue with a check mark
    - If the dropdowns are changed the button returns to a yellow colour with a download symbol
    - If the download fails the button turns red with a disconnect symbol. Change the dropdowns to re-attempt a refresh once the data is available 

There is only one Theme: infectious_disease, but this has been included in a dropdown for visibility that it exists, and to future-proof in case more become available in future. 



In [ ]:
# load the pickle of the tree (if not loaded from file takes nearly an hour to generate)
# was initially part of the APIwrapper class, but moved out as it needed its own processing
with open('full_tree_named.pkl', 'rb') as f:
    root = pickle.load(f)


def getMetric_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    
    # get the data - # todo add error handling for api call
    #leaf_data = requests.get(metric_link, params={ 'page_size': 1800, 'page': 1}).json()
    global df2
    try:
        leaf_data = requests.get(metric_link, params={'page_size': 1800, 'page': 1}).json()
        df2 = wrangle_leaf_data(leaf_data)
        refresh_leaf_graph()
        getMetric_button.icon = "check"
        getMetric_button.description = 'Success'
        getMetric_button.style.button_color = 'lightblue'

    except Exception as err:
        getMetric_button.icon = "unlink"
        getMetric_button.description = "Unavailable"
        getMetric_button.style.button_color = 'red'

        print(f"Data could not be donwloaded: {err}")
    
    # One-off to save json data to file
    # with open("leaf_first.json", "wt") as OUTF:
    #     json.dump(leaf_data, OUTF)

    
getMetric_button=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='solid download'
)

# remember to register your button callback function with the button
getMetric_button.on_click(getMetric_button_callback) # the name of your function inside these brackets




def wrangle_leaf_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    data={}
    for entry in rawdata['results']:
        date=entry['date']
        value=entry['metric_value']
        if date not in data:
            data[date] = value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    leaf_metric_df = pd.DataFrame(index=index, columns=['Metric'])

    for date, value in data.items(): # each entry is a dictionary with cases, admissions and deaths
        pd_date=parse_date(date) # convert to Pandas format
        leaf_metric_df.loc[date, 'Metric'] = value
    leaf_metric_df.fillna(0.0, inplace=True)

    return leaf_metric_df



def plot_metric(change):                                 # revieves the parameter "change" from the widget to refresh, but not used
    fig, ax = plt.subplots(figsize=(14, 6))             
    df2.plot(ax=ax)
    ax.set_title(metrics_dd.label)
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=80)
    ax.tick_params(axis='x', which='major', rotation=80)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    plt.show()

    
# helper dropdown to change without impact, and update graph. Not displayed in control1
helper=wdg.Dropdown(
    options=['One', 'Two'],
    value='One',
    description='helper',
    disabled=False,
)


def refresh_leaf_graph():
    """Change the value of the widget in order to force a redraw of the graph
    since we pull from the API every time with this graph.
    Don't want the useful dropdowns to change as they can trigger a reset and also
    not all dropdowns have >1 choice."""
    current=helper.value
    if current==helper.options[0]:
        other=helper.options[1]
    else:
        other=helper.options[0]
    helper.value=other # forces the redraw
    helper.value=current # now we can change it back



# function to create a dropdown widgets, rather than one by one
def create_dropdown(description, options):
    return wdg.Dropdown(
        options=options,
        value=options[0][1] if options else None,
        description=description,
        disabled=False,
        style={'description_width': '90px'},
        layout=wdg.Layout(width='30%')
    )


# problem with changing earlier dropdowns messing up later dropdown values, e.g. changing sub theme after topics were chosen
# so need to update them all
def update_options(dropdown, options):
    dropdown.options = options
    if options:
        dropdown.value = options[0][1]
    else:
        dropdown.value = None

# update each dropdown
def update_subthemes(*args):
    if theme_dd.value is not None:  # it has a value that might not work with a previous selection (not a problem for theme at the moment but future proofing)
        update_options(subTheme_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children)])
        update_topics()

def update_topics(*args):
    if subTheme_dd.value is not None:
        update_options(topic_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children)])
        update_geotypes()

def update_geotypes(*args):
    if topic_dd.value is not None:
        update_options(geoType_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children)])
        update_geographies()

def update_geographies(*args):
    if geoType_dd.value is not None:
        update_options(geographies_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children)])
        update_metrics()

def update_metrics(*args):
    if geographies_dd.value is not None:
        update_options(metrics_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children)])

# create the dropdown widgets
theme_dd = create_dropdown('1. Theme:', [(child.name, idx) for idx, child in enumerate(root.children)])
subTheme_dd = create_dropdown('2. Sub Theme:', [])
topic_dd = create_dropdown('3. Topic:', [])
geoType_dd = create_dropdown('4. Geo Type:', [])
geographies_dd = create_dropdown('5. Geographies:', [])
metrics_dd = create_dropdown('6. Metrics:', [])


def update_metric_link(*args):
    global metric_link
    metric_link = root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children[metrics_dd.value].link

def reset_button(*args):
    getMetric_button.icon = 'solid download'
    getMetric_button.description = 'Refresh'
    getMetric_button.style.button_color = 'orange'

# couldn't figure out how to do these all in one go, but this readable so not too bad
# observe the dropdowns so downstream resets when a value changes
subTheme_dd.observe(update_topics, names='value')
topic_dd.observe(update_geotypes, names='value')
geoType_dd.observe(update_geographies, names='value')
geographies_dd.observe(update_metrics, names='value')

# observe the dropdowns to update the metric link
subTheme_dd.observe(update_metric_link, names='value')
topic_dd.observe(update_metric_link, names='value')
geoType_dd.observe(update_metric_link, names='value')
geographies_dd.observe(update_metric_link, names='value')
metrics_dd.observe(update_metric_link, names='value')

# observe the dropdowns to reset the button icon
subTheme_dd.observe(reset_button, names='value')
topic_dd.observe(reset_button, names='value')
geoType_dd.observe(reset_button, names='value')
geographies_dd.observe(reset_button, names='value')
metrics_dd.observe(reset_button, names='value')


update_subthemes(None)

controls1 = wdg.VBox([
    wdg.HBox([theme_dd, subTheme_dd, topic_dd], layout=wdg.Layout(justify_content='space-between', width='100%')),
    wdg.HBox([geoType_dd, geographies_dd, metrics_dd], layout=wdg.Layout(justify_content='space-between', width='100%')), 
    wdg.HBox([getMetric_button], layout=wdg.Layout(justify_content='center', width='105%'))
])

# plot first graph from file data
df2=wrangle_leaf_data(leaf_data) # df is the dataframe for plotting

# connect the plotting function and the widget    
graph1 = wdg.interactive_output(plot_metric, {'change': helper})

# actually displays the graph
display(controls1, graph1)


In [ ]:
# running from file so commented out. Kept for future reference.
 
# with open('tree_structure.txt', 'w') as outfile:
#     for pre, fill, node in RenderTree(root):
#         outfile.write("%s%s\n" % (pre, node.name))

#### The tree that was constructed by traversing the API is included below for reference.
  
Due to the number of of NHS trusts and Care Boards, it's quite long!

In [ ]:
with open('tree_structure.txt', 'r') as file:
    content = file.read()

display(Markdown(f"```\n{content}\n```"))